# Processar extrações do [Data Feeds](https://experienceleague.adobe.com/en/docs/analytics/export/ftp-and-sftp/set-up-ftp-accounts/ftp-datafeeds)

Este notebook processa e filtra grandes volumes de dados de arquivos em formato TSV, extraídos do Data Feeds. O código lê os arquivos .tsv alocados em uma pasta de entrada, aplica filtragem com base em valores pré-definidos de uma coluna e salva os resultados em um arquivo CSV em uma pasta de saída. Ele também é otimizado para lidar com arquivos de grandes dimensões por meio de leitura em chunks.

## Configuração dos diretórios e arquivos

In [3]:
import pandas as pd
from pathlib import Path

# Definindo a pasta base a partir da localização do notebook
base_dir = Path.cwd()  # Ou Path().resolve()
print(base_dir)

C:\Users\MarceloMarques\Desktop\python\Jupyter\process_data_feeds_files


In [4]:
# Definindo diretórios de entrada e saída relativos à pasta base
input_dir = base_dir / 'inputs'
output_dir = base_dir / 'outputs'
headers_file = input_dir / 'headers' / 'columns.csv'

# Certifica-se de que o diretório de saída existe
output_dir.mkdir(exist_ok=True)

# Definindo o arquivo de saída
output_file = output_dir / 'filtered_data.csv'

print(input_dir)
print(output_dir)
print(headers_file)
print(output_file)

C:\Users\MarceloMarques\Desktop\python\Jupyter\process_data_feeds_files\inputs
C:\Users\MarceloMarques\Desktop\python\Jupyter\process_data_feeds_files\outputs
C:\Users\MarceloMarques\Desktop\python\Jupyter\process_data_feeds_files\inputs\headers\columns.csv
C:\Users\MarceloMarques\Desktop\python\Jupyter\process_data_feeds_files\outputs\filtered_data.csv


⚠️ Certifique-se de que adicionou corretamente o arquivo com os cabeçalhos na pasta de entrada

In [5]:
# Lendo os cabeçalhos a partir do arquivo columns.csv, ignorando a primeira linha
headers_df = pd.read_csv(headers_file, skiprows=1, header=None)
# Converte o conteúdo da primeira coluna em uma lista
headers = headers_df[0].tolist() 

# Inicializando um DataFrame vazio para armazenar os dados concatenados
concatenated_df = pd.DataFrame(columns=headers)

headers

['date_time',
 'visit_num',
 'visit_page_num',
 'visit_start_pagename',
 'new_visit',
 'visid_timestamp',
 'visid_type',
 'post_visid_type',
 'visit_start_page_url',
 'visid_high',
 'visid_low',
 'mcvisid',
 'post_visid_high',
 'post_visid_low',
 'visid_new',
 'evar113',
 'post_evar113',
 'evar112',
 'post_evar112',
 'evar48',
 'post_evar48',
 'mobiledevice',
 'post_mobiledevice',
 'code_ver',
 'campaign',
 'post_campaign',
 'va_closer_id',
 'pagename',
 'evar1',
 'post_evar1',
 'last_hit_time_gmt']

## Configuração dos filtros
Nesta seção, você pode definir funções de filtragem que serão utilizadas para processar os chunks de dados durante o loop. Cada função de filtro deve receber, no mínimo, o parâmetro `chunk`, que representa uma fração do arquivo de dados a ser processada. Essas funções aplicam as regras de filtragem desejadas e retornam o conjunto de dados filtrado, permitindo que o código seja mais flexível e adaptável a diferentes necessidades de filtragem.

**Edite manualmente o trecho abaixo 👇**

### Variáveis
Informe o nome da coluna que deseja filtrar, e os valores que serão filtrados

In [6]:
# Informe o nome da coluna a ser filtrada
filter_column = 'post_evar48'

# Informe os valores que serão filtrados na coluna. OBS: Os valores devem ser passado como string
filter_values = [
    "9660420",
    "35386133",
    "35399359",
    "36348153",
    "53002692", 
    "61551127",
    "83588016",
    "101994894",
    "104312857"
]
print(filter_column, filter_values)

pagename ['9660420', '35386133', '35399359', '36348153', '53002692', '61551127', '83588016', '101994894', '104312857']


### Funções
Defina funções que serão utilizadas para filtrar os chunks de dados durante o processamento. OBS: Execute apenas a célula da função que de fato fará a filtragem.

In [ ]:
# Função filtrar por valores específicos em uma coluna
def filter_function(chunk):
    # Filtrando as linhas onde a coluna de filtro contém os valores especificados
    return chunk[chunk[filter_column].isin(filter_values)]

In [7]:
# Função para filtrar baseado no número de caracteres de uma coluna
def filter_function(chunk):
    # Exemplo: Filtrando as linhas onde o conteúdo da coluna tem 23 ou mais caracteres
    return chunk[chunk[filter_column].str.len() >= 23]

## Processamento dos arquivos

In [8]:
tsv_files = input_dir.glob('*.tsv')
tsv_files_list = list(tsv_files)  # Converte o iterador em uma lista
print(f"Número de arquivos .tsv encontrados: {len(tsv_files_list)}")

Número de arquivos .tsv encontrados: 4


In [9]:
# Definindo o tamanho dos chunks (ex: 100.000 linhas por vez)
chunk_size = 100000

# Iterando sobre todos os arquivos .tsv na pasta de entrada
for file_path in input_dir.glob('*.tsv'):
    print(f"\nProcessando arquivo: {file_path.name} ...")
    
    # Leitura do arquivo em chunks para evitar sobrecarga de memória. OBS: Todas as colunas são lidas como string
    chunks = pd.read_csv(file_path, sep='\t', header=None, names=headers, chunksize=chunk_size, low_memory=True, dtype=str)

    # Processando cada chunk
    for i, chunk in enumerate(chunks):
        print(f"  ...chunk {i + 1}")  # Informação sobre o chunk atual
        
        # Filtrando as linhas onde a coluna de filtro contém os valores especificados
        filtered_chunk = filter_function(chunk)
        
        # Exibindo quantos resultados foram encontrados no chunk atual
        if len(filtered_chunk) > 0:
            print(f"{len(filtered_chunk)} resultados encontrados")
        
        # Concatenando ao DataFrame principal
        concatenated_df = pd.concat([concatenated_df, filtered_chunk], ignore_index=True)
    
    print(f"Arquivo {file_path.name} processado com sucesso.")


Processando arquivo: 01-bancopansa.production_20240824-010000.tsv ...
  ...chunk 1
1318 resultados encontrados
  ...chunk 2
1257 resultados encontrados
  ...chunk 3
1199 resultados encontrados
  ...chunk 4
1360 resultados encontrados
  ...chunk 5
982 resultados encontrados
  ...chunk 6
1032 resultados encontrados
  ...chunk 7
1218 resultados encontrados
  ...chunk 8
922 resultados encontrados
  ...chunk 9
1308 resultados encontrados
  ...chunk 10
1038 resultados encontrados
  ...chunk 11
982 resultados encontrados
  ...chunk 12
1308 resultados encontrados
  ...chunk 13
859 resultados encontrados
  ...chunk 14
823 resultados encontrados
  ...chunk 15
1315 resultados encontrados
  ...chunk 16
1429 resultados encontrados
  ...chunk 17
1094 resultados encontrados
  ...chunk 18
822 resultados encontrados
  ...chunk 19
1147 resultados encontrados
  ...chunk 20
1350 resultados encontrados
  ...chunk 21
1260 resultados encontrados
  ...chunk 22
1316 resultados encontrados
  ...chunk 23
1212 r

In [10]:
# Salvando o DataFrame concatenado em um arquivo .csv
concatenated_df.to_csv(output_file, index=False)

print(f"\nDados filtrados foram salvos em {output_file}.")


Dados filtrados foram salvos em C:\Users\MarceloMarques\Desktop\python\Jupyter\process_data_feeds_files\outputs\filtered_data.csv.
